In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
from sklearn import model_selection
from sklearn import ensemble
from sklearn import metrics

In [5]:
df_escalacao = pd.read_csv("../Data/Camp_Brasileiro_2024_escalacao.csv")
df_team_stats = pd.read_csv("../Data_Lake/Camp_Brasileiro/2024/Camp_Brasileiro_2024_team_stats_final.csv")
df_players_stats = pd.read_csv("../Data_Lake/Camp_Brasileiro/2024/Camp_Brasileiro_2024_players_stats_final.csv")
df_Games = pd.read_csv("../Data_Lake/Camp_Brasileiro/2024/Camp_Brasileiro_2024_Games_final.csv")

In [10]:
df_stats = df_players_stats

# Lista de games e das medias

In [11]:
lista_games_2024 = df_stats['fixture_id'].unique().tolist()
List_AVG = ['rating','minutes','offsides','shots_total','shots_on','goals_total','goals_conceded','assists','saves','passes_total','passes_key','passes_accuracy','tackles_total','tackles_blocks','tackles_interceptions','duels_total','duels_won','dribbles_attempts','dribbles_success','dribbles_past','fouls_drawn','fouls_committed','cards_yellow','cards_red','penalty_won','penalty_committed','penalty_scored','penalty_missed','penalty_saved']

# Media das estatísticas dos players que estão escalados para jogar a partida de n° fixture_id nos últimos 5 jogos 

In [15]:
def calcular_media_stats(df, fixture_x):
    # Lista de jogadores que participaram do fixture_x
    jogadores_em_x = df[df['fixture_id'] == fixture_x]['player_id'].unique()
    
    # Filtra os jogos anteriores dos jogadores de x
    jogos_anteriores = df[(df['player_id'].isin(jogadores_em_x)) & (df['fixture_id'] < fixture_x)]
    
    # Ordena os jogos por jogador e fixture_id (cronologicamente)
    jogos_ordenados = jogos_anteriores.sort_values(['player_id', 'fixture_id'])
    
    # Seleciona os três últimos jogos de cada jogador
    ultimos_tres = jogos_ordenados.groupby('player_id').tail(5)           #Quantos jogos vou pegar 
    
    # Calcula a média das estatísticas
    media_estatisticas = ultimos_tres.groupby(['player_id','team_id'])[List_AVG].mean().reset_index()
    
    # Garante todos os jogadores de x, mesmo sem jogos anteriores
    resultado_final = pd.DataFrame({'player_id': jogadores_em_x})
    resultado_final = resultado_final.merge(media_estatisticas, on='player_id', how='left')
    
    return resultado_final

In [16]:
# Lista para armazenar os DataFrames de cada fixture
lista_dfs = []

# Iterar sobre cada fixture_id na lista_games_2024
for fixture_x in lista_games_2024:
    # Calcular a média dos últimos três jogos para o fixture atual
    df_medias = calcular_media_stats(df_stats, fixture_x)
    
    # Adicionar a coluna 'fixture_id' ao DataFrame resultante (para identificar o jogo)
    df_medias['fixture_id'] = fixture_x
    
    # Adicionar à lista
    lista_dfs.append(df_medias)

# Concatenar todos os DataFrames da lista em um único df_features
df_features = pd.concat(lista_dfs, ignore_index=True)

- Para cada jogador e jogo, ela calcula a média de suas estatísticas nos últimos 5 jogos anteriores
- As estatísticas são armazenas em df_features que é uma tabela (DataFrame) que resume o desempenho passado de jogadores para cada jogo (fixture) 

In [17]:
df_features

,player_id,team_id,rating,minutes,offsides,shots_total,shots_on,goals_total,goals_conceded,assists,saves,passes_total,passes_key,passes_accuracy,tackles_total,tackles_blocks,tackles_interceptions,duels_total,duels_won,dribbles_attempts,dribbles_success,dribbles_past,fouls_drawn,fouls_committed,cards_yellow,cards_red,penalty_won,penalty_committed,penalty_scored,penalty_missed,penalty_saved,fixture_id
0,50077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1180355
1,6080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1180355
2,9909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1180355
3,452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1180355
4,10163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1180355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17370,403306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1180733
17371,44436,133.0,1.06,0.600000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.800000,0.0,0.8,0.000000,0.0,0.0,0.400000,0.2,0.200000,0.200000,0.000000,0.0,0.2,0.000000,0.2,0.0,0.0,0.0,0.0,0.0,1180733
17372,2563,133.0,1.30,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.000000,0.0,1.8,0.000000,0.0,0.2,0.400000,0.2,0.200000,0.000000,0.000000,0.2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1180733
17373,454139,133.0,2.10,3.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.333333,0.0,0.0,2.666667,1.0,0.666667,0.333333,0.333333,0.0,1.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,1180733


In [28]:
#pd.merge(df_features,df_Games)
df_Games = df_Games.rename(columns={'id_partida': 'fixture_id'})

In [29]:
df_jogos = df_Games[['fixture_id', 'Home_Team','Away_Team','home.id','away.id']]
df_form = df_escalacao[['fixture_id','player_id','team']]

In [30]:
df_home_away = pd.merge(df_form,df_jogos, on='fixture_id',how='inner')

In [31]:
df_features

,player_id,team_id,rating,minutes,offsides,shots_total,shots_on,goals_total,goals_conceded,assists,saves,passes_total,passes_key,passes_accuracy,tackles_total,tackles_blocks,tackles_interceptions,duels_total,duels_won,dribbles_attempts,dribbles_success,dribbles_past,fouls_drawn,fouls_committed,cards_yellow,cards_red,penalty_won,penalty_committed,penalty_scored,penalty_missed,penalty_saved,fixture_id
0,50077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1180355
1,6080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1180355
2,9909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1180355
3,452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1180355
4,10163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1180355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17370,403306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1180733
17371,44436,133.0,1.06,0.600000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.800000,0.0,0.8,0.000000,0.0,0.0,0.400000,0.2,0.200000,0.200000,0.000000,0.0,0.2,0.000000,0.2,0.0,0.0,0.0,0.0,0.0,1180733
17372,2563,133.0,1.30,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.000000,0.0,1.8,0.000000,0.0,0.2,0.400000,0.2,0.200000,0.000000,0.000000,0.2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1180733
17373,454139,133.0,2.10,3.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.333333,0.0,0.0,2.666667,1.0,0.666667,0.333333,0.333333,0.0,1.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,1180733


In [32]:
#Vai dizer se está jogando em casa ou fora de casa
df_home_away['condicao'] = df_home_away['team'] == df_home_away['Home_Team']

In [33]:
df_var = pd.merge(df_features,df_home_away, on = ['fixture_id','player_id'])

In [34]:
#Media das estatisticas dos jogadores nos ultimos 5 jogos 
df_var = df_var.groupby(['fixture_id','team'])[List_AVG].mean().reset_index()

In [36]:
df_var.dropna(inplace=True)

In [37]:
#Tabela de estatistica de players completa
df_var

,fixture_id,team,rating,minutes,offsides,shots_total,shots_on,goals_total,goals_conceded,assists,saves,passes_total,passes_key,passes_accuracy,tackles_total,tackles_blocks,tackles_interceptions,duels_total,duels_won,dribbles_attempts,dribbles_success,dribbles_past,fouls_drawn,fouls_committed,cards_yellow,cards_red,penalty_won,penalty_committed,penalty_scored,penalty_missed,penalty_saved
20,1180365,Atletico Paranaense,5.027273,41.954545,0.090909,0.545455,0.318182,0.136364,0.000000,0.181818,0.000000,24.727273,0.727273,21.681818,0.636364,0.000000,0.454545,3.363636,2.000000,0.590909,0.363636,0.090909,0.409091,0.636364,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
21,1180365,Gremio,4.471429,44.333333,0.047619,0.285714,0.095238,0.047619,0.000000,0.000000,0.047619,23.333333,0.333333,20.428571,0.571429,0.238095,0.380952,5.238095,2.190476,0.761905,0.333333,0.666667,0.761905,0.428571,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
22,1180366,Atletico-MG,4.276190,39.095238,0.095238,0.238095,0.142857,0.000000,0.000000,0.000000,0.095238,15.666667,0.190476,12.380952,0.476190,0.142857,0.380952,4.142857,2.190476,0.714286,0.333333,0.428571,0.714286,0.476190,0.142857,0.000000,0.000000,0.000000,0.0,0.000000,0.0
23,1180366,Criciuma,4.710000,43.000000,0.050000,0.200000,0.100000,0.000000,0.050000,0.050000,0.150000,19.650000,0.350000,15.900000,0.600000,0.200000,0.550000,4.750000,2.250000,1.100000,0.650000,0.850000,0.550000,0.450000,0.200000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
24,1180367,Bahia,4.540909,45.000000,0.090909,0.318182,0.181818,0.045455,0.090909,0.045455,0.318182,21.227273,0.272727,17.545455,0.454545,0.090909,0.409091,5.363636,2.090909,0.818182,0.363636,0.318182,0.909091,0.590909,0.136364,0.000000,0.000000,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753,1180732,Internacional,4.280000,37.980952,0.076190,0.380952,0.190476,0.038095,0.047619,0.057143,0.085714,16.876190,0.276190,13.857143,0.809524,0.076190,0.476190,3.828571,1.838095,0.638095,0.266667,0.323810,0.390476,0.457143,0.085714,0.000000,0.009524,0.000000,0.0,0.000000,0.0
754,1180733,Cuiaba,3.134444,28.855556,0.022222,0.188889,0.088889,0.033333,0.000000,0.011111,0.000000,8.211111,0.151852,6.500000,0.544444,0.133333,0.211111,2.944444,1.455556,0.511111,0.233333,0.211111,0.322222,0.311111,0.100000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
755,1180733,Vasco DA Gama,4.116190,37.134921,0.076190,0.295238,0.114286,0.047619,0.085714,0.038095,0.219048,16.030159,0.185714,13.833333,0.520635,0.133333,0.304762,2.912698,1.576190,0.307937,0.187302,0.196825,0.419048,0.419048,0.111111,0.019048,0.000000,0.000000,0.0,0.000000,0.0
756,1180734,Cruzeiro,3.843478,37.350000,0.043478,0.417391,0.130435,0.043478,0.069565,0.017391,0.226087,18.532609,0.436957,16.239130,0.400000,0.086957,0.219565,3.039130,1.358696,0.619565,0.243478,0.278261,0.393478,0.426087,0.078261,0.000000,0.000000,0.000000,0.0,0.000000,0.0


In [38]:
# Supondo que você tem seus DataFrames df_var e df_Games

# Inicialize uma lista vazia para armazenar os valores de 'home' (True ou False)
home_values = []

# Itere sobre cada linha do DataFrame df_var
for index, row_var in df_var.iterrows():
    fixture_id_var = row_var['fixture_id']
    team_var = row_var['team']

    # Encontre a linha correspondente em df_Games com o mesmo fixture_id
    # Usamos .loc para selecionar a linha onde 'fixture_id' é igual a fixture_id_var
    # Como esperamos apenas uma linha correspondente por fixture_id em df_Games (formato comum de tabela de jogos)
    # podemos usar .iloc[0] para pegar a primeira (e única esperada) linha como uma Series
    games_row = df_Games.loc[df_Games['fixture_id'] == fixture_id_var]

    # Verifique se encontramos uma linha correspondente em df_Games
    if not games_row.empty:
        # Obtém o 'Home_Team' do df_Games para este fixture_id
        home_team_games = games_row['Home_Team'].iloc[0] # .iloc[0] para pegar o valor da Series

        # Compara o 'team' de df_var com o 'Home_Team' de df_Games
        is_home = (team_var == home_team_games)
        home_values.append(is_home) # Adiciona True ou False à lista
    else:
        # Se não encontrou correspondência em df_Games, você pode decidir o que fazer
        # Por exemplo, pode adicionar False ou NaN, dependendo da sua lógica.
        # Aqui, estou adicionando False como padrão, indicando que não é considerado 'home' se não encontrado.
        home_values.append(False) # Ou poderia ser home_values.append(None) para NaN

# Adiciona a lista de valores 'home' como uma nova coluna no df_var
df_var['home'] = home_values

In [39]:
df_var

,fixture_id,team,rating,minutes,offsides,shots_total,shots_on,goals_total,goals_conceded,assists,saves,passes_total,passes_key,passes_accuracy,tackles_total,tackles_blocks,tackles_interceptions,duels_total,duels_won,dribbles_attempts,dribbles_success,dribbles_past,fouls_drawn,fouls_committed,cards_yellow,cards_red,penalty_won,penalty_committed,penalty_scored,penalty_missed,penalty_saved,home
20,1180365,Atletico Paranaense,5.027273,41.954545,0.090909,0.545455,0.318182,0.136364,0.000000,0.181818,0.000000,24.727273,0.727273,21.681818,0.636364,0.000000,0.454545,3.363636,2.000000,0.590909,0.363636,0.090909,0.409091,0.636364,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,False
21,1180365,Gremio,4.471429,44.333333,0.047619,0.285714,0.095238,0.047619,0.000000,0.000000,0.047619,23.333333,0.333333,20.428571,0.571429,0.238095,0.380952,5.238095,2.190476,0.761905,0.333333,0.666667,0.761905,0.428571,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,True
22,1180366,Atletico-MG,4.276190,39.095238,0.095238,0.238095,0.142857,0.000000,0.000000,0.000000,0.095238,15.666667,0.190476,12.380952,0.476190,0.142857,0.380952,4.142857,2.190476,0.714286,0.333333,0.428571,0.714286,0.476190,0.142857,0.000000,0.000000,0.000000,0.0,0.000000,0.0,True
23,1180366,Criciuma,4.710000,43.000000,0.050000,0.200000,0.100000,0.000000,0.050000,0.050000,0.150000,19.650000,0.350000,15.900000,0.600000,0.200000,0.550000,4.750000,2.250000,1.100000,0.650000,0.850000,0.550000,0.450000,0.200000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,False
24,1180367,Bahia,4.540909,45.000000,0.090909,0.318182,0.181818,0.045455,0.090909,0.045455,0.318182,21.227273,0.272727,17.545455,0.454545,0.090909,0.409091,5.363636,2.090909,0.818182,0.363636,0.318182,0.909091,0.590909,0.136364,0.000000,0.000000,0.000000,0.0,0.000000,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753,1180732,Internacional,4.280000,37.980952,0.076190,0.380952,0.190476,0.038095,0.047619,0.057143,0.085714,16.876190,0.276190,13.857143,0.809524,0.076190,0.476190,3.828571,1.838095,0.638095,0.266667,0.323810,0.390476,0.457143,0.085714,0.000000,0.009524,0.000000,0.0,0.000000,0.0,False
754,1180733,Cuiaba,3.134444,28.855556,0.022222,0.188889,0.088889,0.033333,0.000000,0.011111,0.000000,8.211111,0.151852,6.500000,0.544444,0.133333,0.211111,2.944444,1.455556,0.511111,0.233333,0.211111,0.322222,0.311111,0.100000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,True
755,1180733,Vasco DA Gama,4.116190,37.134921,0.076190,0.295238,0.114286,0.047619,0.085714,0.038095,0.219048,16.030159,0.185714,13.833333,0.520635,0.133333,0.304762,2.912698,1.576190,0.307937,0.187302,0.196825,0.419048,0.419048,0.111111,0.019048,0.000000,0.000000,0.0,0.000000,0.0,False
756,1180734,Cruzeiro,3.843478,37.350000,0.043478,0.417391,0.130435,0.043478,0.069565,0.017391,0.226087,18.532609,0.436957,16.239130,0.400000,0.086957,0.219565,3.039130,1.358696,0.619565,0.243478,0.278261,0.393478,0.426087,0.078261,0.000000,0.000000,0.000000,0.0,0.000000,0.0,False


# Construcao Variável Resposta

In [40]:
df_Games['Total_Gols'] = df_Games['Gols_Home'] + df_Games['Gols_Away']
df_Games

,fixture_id,id_league,referee,date,Stadium,Home_Team,home.id,Away_Team,away.id,home.winner,Gols_Home,Gols_Away,Half_Time_Gols_Home,Half_Time_Gols_Away,Empate,Total_Gols
0,1180355,71,Rodrigo José Pereira de Lima,2024-04-13T21:30:00+00:00,Estádio José Pinheiro Borda,Internacional,119,Bahia,118,True,2,1,0,0,0,3
1,1180364,71,Bruno Pereira Vasconcelos,2024-04-13T21:30:00+00:00,Estádio Heriberto Hülse,Criciuma,140,Juventude,152,False,1,1,1,0,1,2
2,1180358,71,Maguielson Lima Barbosa,2024-04-14T00:00:00+00:00,Estadio Jornalista Mário Filho,Fluminense,124,RB Bragantino,794,False,2,2,1,0,1,4
3,1180361,71,Alex Gomes,2024-04-14T00:00:00+00:00,MorumBIS,Sao Paulo,126,Fortaleza EC,154,False,1,2,0,0,0,3
4,1180360,71,Yuri Elino Ferreira da Cruz,2024-04-14T19:00:00+00:00,Neo Química Arena,Corinthians,131,Atletico-MG,1062,False,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,1180734,71,Edina Alves Batista,2024-12-08T19:00:00+00:00,Estádio Alfredo Jaconi,Juventude,152,Cruzeiro,135,False,0,1,0,1,0,1
376,1180732,71,Matheus Delgado Candançan,2024-12-08T19:00:00+00:00,Estádio Governador Plácido Aderaldo Castelo,Fortaleza EC,154,Internacional,119,True,3,0,2,0,0,3
377,1180731,71,Sávio Pereira,2024-12-08T19:00:00+00:00,Estádio Nabi Abi Chedid,RB Bragantino,794,Criciuma,140,True,5,1,2,1,0,6
378,1180726,71,Rafael Rodrigo Klein,2024-12-08T19:00:00+00:00,Arena MRV,Atletico-MG,1062,Atletico Paranaense,134,True,1,0,0,0,0,1


# Modelo de Test

In [34]:
features = df.columns[0:-2].tolist()
target = 'Total_Gols'

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = model_selection.train_test_split(df[features],df[target], test_size= 0.2, random_state=42, stratify=df[target]) 

In [35]:
modelo = ensemble.RandomForestClassifier()
modelo.fit(X_train, y_train)


#---
y_train_pred = modelo.predict(X_train)
y_train_prob = modelo.predict_proba(X_train)
y_test_pred = modelo.predict(X_test)
y_test_prob = modelo.predict_proba(X_test)
print(f'Acurácia treino: {metrics.accuracy_score(y_train, y_train_pred):.4f}')
print(f'Acurácia teste: {metrics.accuracy_score(y_test, y_test_pred):.4f}')
print(f'AUC treino: {metrics.roc_auc_score(y_train, y_train_prob[:,1]):.4f}')
print(f'AUC teste: {metrics.roc_auc_score(y_test, y_test_prob[:,1]):.4f}')

print(f'Proporção de target=1 (treino): {y_train.mean():.2%}')
print(f'Proporção de target=1 (teste): {y_test.mean():.2%}')

Acurácia treino: 1.0000
Acurácia teste: 0.7162
AUC treino: 1.0000
AUC teste: 0.5903
Proporção de target=1 (treino): 72.35%
Proporção de target=1 (teste): 72.97%
